### Getting started: 4 steps to DeepCTR Estimator with TFRecord


### Step 1: Import model

In [ ]:
import tensorflow as tf
from deepctr.estimator.inputs import input_fn_tfrecord
from deepctr.estimator.models import WDLEstimator


### Step 2: Generate feature columns for linear part and dnn part

In [3]:
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

dnn_feature_columns = []
linear_feature_columns = []

In [4]:
for i, feat in enumerate(sparse_features):
    dnn_feature_columns.append(
        tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_identity(feat, 1000), 4)
    )
    linear_feature_columns.append(tf.feature_column.categorical_column_with_identity(feat, 1000))
for feat in dense_features:
    dnn_feature_columns.append(tf.feature_column.numeric_column(feat))
    linear_feature_columns.append(tf.feature_column.numeric_column(feat))

In [5]:
print(dnn_feature_columns)

[EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='C1', number_buckets=1000, default_value=None), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x64144cb10>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True), EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='C2', number_buckets=1000, default_value=None), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x64144c210>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True), EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='C3', number_buckets=1000, default_value=None), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x64144c550>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True), EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='C4', n

### Step 3: Generate the training samples with TFRecord format

In [6]:
feature_description = {k: tf.io.FixedLenFeature(dtype=tf.int64, shape=1) for k in sparse_features}
feature_description.update(
    {k: tf.io.FixedLenFeature(dtype=tf.float32, shape=1) for k in dense_features}
)
feature_description['label'] = tf.io.FixedLenFeature(dtype=tf.float32, shape=1)

train_model_input = input_fn_tfrecord('./criteo_sample.tr.tfrecords', feature_description, 'label', batch_size=256,
                                      num_epochs=1, shuffle_factor=10)
test_model_input = input_fn_tfrecord('./criteo_sample.te.tfrecords', feature_description, 'label',
                                     batch_size=2 ** 14, num_epochs=1, shuffle_factor=0)

### Step 4: Train and evaluate the model

In [7]:
model = WDLEstimator(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=[4, 4], dnn_dropout=0.5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/n2/6bkx5wwj5zn_dpld14gmpk0c0000gp/T/tmpvcmb4056', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x641261dd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
model.train(train_model_input)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


AttributeError: in converted code:

    /Users/taoyizhou/yizhou/git/DeepCTR/deepctr/estimator/inputs.py:26 _parse_examples  *
        features = tf.parse_single_example(serial_exmp, features=feature_description)

    AttributeError: module 'tensorflow' has no attribute 'parse_single_example'


In [8]:
eval_result = model.evaluate(test_model_input)

print(eval_result)

INFO:tensorflow:Could not find trained model in model_dir: /var/folders/n2/6bkx5wwj5zn_dpld14gmpk0c0000gp/T/tmpf377vlry, running initialization to evaluate.


AttributeError: in converted code:

    /Users/taoyizhou/yizhou/git/DeepCTR/deepctr/estimator/inputs.py:26 _parse_examples  *
        features = tf.parse_single_example(serial_exmp, features=feature_description)

    AttributeError: module 'tensorflow' has no attribute 'parse_single_example'
